In [1]:
import numpy as np
import torch
import pygame
import random
from torch import nn
from torch.distributions import Categorical
import torch.nn.functional as F
import copy
from collections import deque
import tqdm

colors = [
    (0, 0, 0),
    (120, 37, 179),
    (100, 179, 179),
    (80, 34, 22),
    (80, 134, 22),
    (180, 34, 22),
    (180, 34, 122),
]


pygame 2.1.0 (SDL 2.0.16, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
class Event():
    type = None
    key = None

    def __init__(self, type, key):
        self.type = type
        self.key = key

In [4]:
class Figure:
    x = 0
    y = 0

    figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],
        [[4, 5, 9, 10], [2, 6, 5, 9]],
        [[6, 7, 9, 10], [1, 5, 6, 10]],
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        [[1, 2, 5, 6]],
    ]

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(self.figures) - 1)
        self.color = 1
        self.rotation = 0

    def image(self):
        return self.figures[self.type][self.rotation]

    def rotate(self):
        self.rotation = (self.rotation + 1) % len(self.figures[self.type])


In [5]:
class Tetris:
    def __init__(self, height, width):
        self.level = 2
        self.score = 0
        self.field = []
        self.height = 0
        self.width = 0
        self.x = 100
        self.y = 60
        self.zoom = 20
        self.figure = None
    
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.done = False
        for i in range(height):
            new_line = []
            for j in range(width):
                new_line.append(0)
            self.field.append(new_line)
            
    def reset(self):
        self.field = []
        self.score = 0
        self.done = False
        for i in range(self.height):
            new_line = []
            for j in range(self.width):
                new_line.append(0)
            self.field.append(new_line)

    def new_figure(self):
        self.figure = Figure(3, 0)

    def intersects(self, figure):
        intersection = False
        for i in range(4):
            for j in range(4):
                if i * 4 + j in figure.image():
                    if i + figure.y > self.height - 1 or \
                            j + figure.x > self.width - 1 or \
                            j + figure.x < 0 or \
                            self.field[i + figure.y][j + figure.x] > 0:
                        intersection = True
        return intersection
    
    def step(self, figure):  
        state = copy.deepcopy(self.field)
        for i in range(4):
            for j in range(4):
                if i * 4 + j in figure.image():
                    state[i + figure.y][j + figure.x] = figure.color
#         print("/////////////////////////")
#         for row in state:
#             print(row)
        return state

    def break_lines(self):
        lines = 0
        for i in range(1, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1 - 1][j]
        self.score += lines ** 2

    def go_space(self):
        while not self.intersects(self.figure):
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects(self.figure):
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.new_figure()
        self.done =  self.intersects(self.figure)

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        intersects = self.intersects(self.figure)
        if intersects:
            self.figure.x = old_x
        return intersects

    def rotate(self):
        old_rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects(self.figure):
            self.figure.rotation = old_rotation


In [6]:
class NeuralNet(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super(NeuralNet, self).__init__()

        """ CODE HERE:
                Implement the neural network here
        """
        self.fc1 = nn.Linear(input_dims, hidden_dims)
        self.fc2 = nn.Linear(hidden_dims, hidden_dims)
        self.fc3 = nn.Linear(hidden_dims, output_dims)

    def forward(self, x):
        """ CODE HERE:
                Implement the forward propagation
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [13]:
def get_state_properties(state):
    tower_h = []
    holes = []
    height_board = len(state)
    width_board  = len(state[0])
    lines = [1]*len(state)
    for j in range(width_board):
        tower_h.append(0)
        holes.append(0)
        top_reached = False
        for i in range(height_board):
            if state[i][j] == 0:
                if top_reached:
                    holes[j] += 1
                if lines[i] == 1:
                    lines[i] = 0
            elif state[i][j] != 0 and not top_reached:
                tower_h[j] = height_board - i
                top_reached =True
    bumps = []
    for i in range(width_board):
        bumps.append(0)
        if i != width_board-1:
            bumps[i] += abs(tower_h[i+1]-tower_h[i])
        if i != 0:
            bumps[i] += abs(tower_h[i-1]-tower_h[i])
    tower_h = np.array(tower_h) - sum(lines)
    feature = torch.tensor([sum(bumps), sum(holes), sum(tower_h), max(tower_h), sum(lines)], dtype=torch.float32)
    return feature, sum(lines)*10


def get_bfs_score(state):
    feature, _ = get_state_properties(state)
    return -feature[0].item() - 13*feature[1].item()  - feature[3].item()

def simulate(t):
    fig = Figure(3, 0)
    fig.type = t.figure.type
    fig.color = t.figure.color
    opt = float("-inf")
    opt_rotation, opt_x = 0, fig.x
    if t.intersects(fig):
        return opt_rotation, opt_x
    fig.x = -3
    for i in range(t.width + 3):
        for j in range(len(fig.figures[fig.type])):
            if not t.intersects(fig):
                while not t.intersects(fig):
                    fig.y += 1
                fig.y-=1
                possible_state = t.step(fig)
#                 score = get_bfs_score(possible_state)
                
                feature, _ = get_state_properties(possible_state)
                score = model(feature)
                if score > opt:
                    opt = score
                    opt_rotation = fig.rotation
                    opt_x = fig.x
                fig.y = 0
            fig.rotate()
        fig.x += 1
    return opt_rotation, opt_x


def run_ai(t):
    rotation, x = simulate(t)
    if t.figure.rotation != rotation:
        return [Event(pygame.KEYDOWN, pygame.K_UP)]
    elif t.figure.x < x:
        return [Event(pygame.KEYDOWN, pygame.K_RIGHT)]
    elif t.figure.x > x:
        return [Event(pygame.KEYDOWN, pygame.K_LEFT)]
    else:
        return [Event(pygame.KEYDOWN, pygame.K_SPACE)]
    return []

In [22]:
feature_len = 5
model = NeuralNet(feature_len,64,1)

# Preffered way
FILE = "model.pth"
torch.save(model, FILE)

In [ ]:
# feature_len = 5
# FILE = "model.pth"
# # model must be created agin with parameters
# model = torch.load(FILE)
# model.eval()

width = 10
height = 20

env = Tetris(height, width)

learning_rate = 5e-4
gamma = 0.99
epsilon = 1
eps_dec = 1e-4
eps_min = 0.01
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
replay_memory = deque(maxlen=30000)
scores = deque(maxlen=100)
n_episodes = 100000
batch_size = 500

def simulate_RL(t, isRandom):
    fig = Figure(3, 0)
    reward, best_state, best_feature, done = -300, None, torch.zeros(feature_len), True
    if t.intersects(fig):
        return best_state, best_feature, reward, done
    fig.x = -3
    fig.type = t.figure.type
    fig.color = t.figure.color
    opt = float("-inf")
    states, features, rewards = [], [], []
    for i in range(t.width + 3):
        for j in range(len(fig.figures[fig.type])):
            if not t.intersects(fig):
                while not t.intersects(fig):
                    fig.y += 1
                fig.y-=1
                done = False
                state = t.step(fig)
                feature, r = get_state_properties(state)
                score = model(feature)
                if score > opt:
                    opt = score
                    reward = r
                    best_state = state
                    best_feature = feature
                states.append(state)
                features.append(feature)
                rewards.append(r)
                fig.y = 0
            fig.rotate()
        fig.x += 1
    if isRandom and not done:
        index = random.randint(0, len(states) - 1)
        return states[index], features[index], rewards[index], done
    return best_state, best_feature, reward, done

pbar = tqdm.trange(n_episodes)
for t in pbar:
    env.reset()
    feature = torch.zeros(feature_len)
    epsilon =  epsilon - eps_dec \
            if epsilon > eps_min else eps_min
    score = 0
    while True:
        random_action =  random.random() <= epsilon
        env.new_figure()
        next_state, next_feature, reward, done = simulate_RL(env, random_action)
        score += reward
        
        replay_memory.append((feature, reward, next_feature, done))
        feature = next_feature
        batch = random.sample(replay_memory, min(len(replay_memory), batch_size))
        feature_batch, reward_batch, next_feature_batch, done_batch = zip(*batch)
        feature_batch = torch.stack(tuple(feat for feat in feature_batch))
        reward_batch = torch.from_numpy(np.array(reward_batch, dtype=np.float32)[:, None])
        next_feature_batch = torch.stack(tuple(feat for feat in next_feature_batch))

        q_values = model(feature_batch)
        with torch.no_grad():
            next_prediction_batch = model(next_feature_batch)

        y_batch = torch.cat(
                tuple(reward if done else reward + 1 + gamma * prediction for reward, done, prediction in
                      zip(reward_batch, done_batch, next_prediction_batch)))[:, None]
        optimizer.zero_grad()
        loss = criterion(q_values, y_batch)
        loss.backward()
        optimizer.step()
        if done:
            scores.append(score)
#             print("///////////Game Over//////////////////")
            break
#         print()
#         for row in env.field:
#             print(row)
#         print(next_feature)
        env.field = next_state
        env.break_lines()
    if t%100 ==0:
        print(np.mean(scores), epsilon)
    if t%500 == 0:
        torch.save(model, FILE)


  0%|                                                                             | 1/100000 [00:00<8:07:19,  3.42it/s]

-300.0 0.9999


  0%|                                                                          | 101/100000 [00:49<15:09:37,  1.83it/s]

-299.7 0.9899000000000011


  0%|▏                                                                         | 201/100000 [01:43<16:45:13,  1.65it/s]

...

 10%|██████▊                                                               | 9801/100000 [2:53:43<137:42:47,  5.50s/it]

316.5 0.019900000000093752


 10%|██████▉                                                               | 9901/100000 [3:03:04<119:48:48,  4.79s/it]

619.1 0.009900000000093813


 10%|██████▉                                                               | 9972/100000 [3:08:49<180:51:34,  7.23s/it]

In [20]:
n_episodes = 1
pbar = tqdm.trange(n_episodes)
for t in pbar:
    env.reset() 
    feature = torch.zeros(feature_len).float()
    epsilon =  epsilon - eps_dec \
            if epsilon > eps_min else eps_min
    score = 0
    while True:
        random_action =  random.random() <= epsilon
        env.new_figure()
        next_state, next_feature, reward, done = simulate_RL(env, random_action)
        score += reward
        replay_memory.append((feature, reward, next_feature, done))
        if done:
            scores.append(score)
            print("///////////Game Over//////////////////")
            break
        env.field = next_state
        env.break_lines()
        print()
        for row in env.field:
            print(row)
        print(next_feature)
        feature = next_feature
print(score)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([4., 0., 4., 2., 0.])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0

tensor([32., 17., 77., 14.,  0.])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 0, 1, 1, 1, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
tensor([32., 17., 81., 14.,  0.])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.98it/s]

tensor([ 14.,  25., 131.,  15.,   0.])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 0, 0, 0, 1, 1, 1, 0]
[1, 1, 1, 0, 0, 0, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 0]
[1, 0, 1, 1, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 0, 0, 1, 1, 1, 1]
tensor([ 10.,  26., 136.,  15.,   0.])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 0, 0, 0, 1,

In [37]:
t = Tetris(20,10)

while True:
    sentence = input("Input:")
    if sentence =="q":
        break
    if t.figure is None:
        t.new_figure()
    if sentence == "u":
        t.rotate()
    elif sentence == "l":
        t.go_side(-1)
    elif sentence == "r":
        t.go_side(1)
    elif sentence == "s":
        t.go_space()
    print(simulate(t))
    t.go_down()
    print()
    for row in t.field:
        print(row)

Input:q


In [21]:
# Initialize the game engine
pygame.init()

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)

size = (400, 500)
screen = pygame.display.set_mode(size)

pygame.display.set_caption("Tetris")

# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
fps = 10
game = Tetris(20, 10)
counter = 0

pressing_down = False

while not done:
    if game.figure is None:
        game.new_figure()
    counter += 1
    if counter > 100000:
        counter = 0

    if counter % (fps // game.level // 2) == 0 or pressing_down:
        if not game.done:
            game.go_down()

    for event in list(pygame.event.get()) + run_ai(game):
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                game.rotate()
            if event.key == pygame.K_DOWN:
                pressing_down = True
            if event.key == pygame.K_LEFT:
                game.go_side(-1)
            if event.key == pygame.K_RIGHT:
                game.go_side(1)
            if event.key == pygame.K_SPACE:
                game.go_space()
            if event.key == pygame.K_ESCAPE:
                game.__init__(20, 10)

    if event.type == pygame.KEYUP:
            if event.key == pygame.K_DOWN:
                pressing_down = False

    screen.fill(WHITE)

    for i in range(game.height):
        for j in range(game.width):
            pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
            if game.field[i][j] > 0:
                pygame.draw.rect(screen, colors[game.field[i][j]],
                                 [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])
    
    if game.figure is not None:
        for i in range(4):
            for j in range(4):
                p = i * 4 + j
                if p in game.figure.image():
                    pygame.draw.rect(screen, colors[game.figure.color],
                                     [game.x + game.zoom * (j + game.figure.x) + 1,
                                      game.y + game.zoom * (i + game.figure.y) + 1,
                                      game.zoom - 2, game.zoom - 2])

    font = pygame.font.SysFont('Calibri', 25, True, False)
    font1 = pygame.font.SysFont('Calibri', 65, True, False)
    text = font.render("Score: " + str(game.score), True, BLACK)
    text_game_over = font1.render("Game Over", True, (255, 125, 0))
    text_game_over1 = font1.render("Press ESC", True, (255, 215, 0))

    screen.blit(text, [0, 0])
    if game.done:
        screen.blit(text_game_over, [20, 200])
        screen.blit(text_game_over1, [25, 265])

    pygame.display.flip()
    clock.tick(fps)

pygame.quit()